# Getting set up

## Setting up the notebook

In [ ]:
### Installing dependencies
!pip install openai

!apt-get update
!apt-get install -y iverilog

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.8 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,388 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [

## Use LLM to generate Verilog code from natural language description

### Example 1: binary_to_bcd

In [ ]:
! mkdir -p binary_to_bcd
! cd binary_to_bcd && curl -O https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/binary_to_bcd_tb.v

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1078  100  1078    0     0   1512      0 --:--:-- --:--:-- --:--:--  1511


In [ ]:
verilog_generation_prompt = '''
I am trying to create a Verilog model binary_to_bcd_converter for a binary to binary-coded-decimal converter. It must meet the following specifications:
	- Inputs:
		- Binary input (5-bits)
	- Outputs:
		- BCD (8-bits: 4-bits for the 10's place and 4-bits for the 1's place)

How would I write a design that meets these specifications?

binary_to_bcd_tb.v:8: error: Unknown module type: binary_to_bcd_converter
2 error(s) during elaboration.
*** These modules were missing:
        binary_to_bcd_converter referenced 1 times.
***

binary_to_bcd_tb.v:8: error: port ``binary_input'' is not a port of uut.
binary_to_bcd_tb.v:8: error: port ``bcd_output'' is not a port of uut.
2 error(s) during elaboration.

Testing Binary-to-BCD Converter...
VCD info: dumpfile my_design.vcd opened for output.
Error: Test case 10 failed. Expected BCD: 8'b10000, Got: 8'b100000
'''

Now you can use the openai library to generate text from your prompt.

Before using LLMs for Verilog generation, you have to first get the api-key:

openai llms: https://platform.openai.com/api-keys
deepseek llms: https://platform.deepseek.com/sign_in
free llms provided by NVIDIA: https://build.nvidia.com/explore/discover

In [ ]:
from openai import OpenAI
import os
client = OpenAI(base_url="https://openrouter.ai/api/v1",
    api_key = "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"
)

completion = client.chat.completions.create(
  model = "arcee-ai/trinity-large-preview:free",
  messages=[{"role":"user","content":verilog_generation_prompt}],
  max_tokens=129000, # limits the maximum number of tokens (words or pieces of words) that the model will generate in its response.
  stream=False
)

print(completion.choices[0].message.content)

Based on your error messages, you're missing the module definition and the port names don't match. Here's a complete implementation that should work:

```verilog
module binary_to_bcd_converter (
    input [4:0] binary_input,
    output [7:0] bcd_output
);

// Temporary variables to hold BCD digits
reg [3:0] tens;
reg [3:0] ones;

// Combinational logic to convert binary to BCD
always @(*) begin
    tens = 4'd0;
    ones = 4'd0;
    
    case (binary_input)
        5'd0:   {tens, ones} = 8'b00000000;
        5'd1:   {tens, ones} = 8'b00000001;
        5'd2:   {tens, ones} = 8'b00000010;
        5'd3:   {tens, ones} = 8'b00000011;
        5'd4:   {tens, ones} = 8'b00000100;
        5'd5:   {tens, ones} = 8'b00000101;
        5'd6:   {tens, ones} = 8'b00000110;
        5'd7:   {tens, ones} = 8'b00000111;
        5'd8:   {tens, ones} = 8'b00001000;
        5'd9:   {tens, ones} = 8'b00001001;
        5'd10:  {tens, ones} = 8'b00010000;
        5'd11:  {tens, ones} = 8'b00010001;
        5'd

Next, extract the generated verilog code from LLM response.

In [ ]:
output_verilog_code = '''
module binary_to_bcd_converter (
    input [4:0] binary_input,
    output [7:0] bcd_output
);

// Temporary variables to hold BCD digits
reg [3:0] tens;
reg [3:0] ones;

// Combinational logic to convert binary to BCD
always @(*) begin
    tens = 4'd0;
    ones = 4'd0;

    case (binary_input)
        5'd0:   {tens, ones} = 8'b00000000;
        5'd1:   {tens, ones} = 8'b00000001;
        5'd2:   {tens, ones} = 8'b00000010;
        5'd3:   {tens, ones} = 8'b00000011;
        5'd4:   {tens, ones} = 8'b00000100;
        5'd5:   {tens, ones} = 8'b00000101;
        5'd6:   {tens, ones} = 8'b00000110;
        5'd7:   {tens, ones} = 8'b00000111;
        5'd8:   {tens, ones} = 8'b00001000;
        5'd9:   {tens, ones} = 8'b00001001;
        5'd10:  {tens, ones} = 8'b00010000;
        5'd11:  {tens, ones} = 8'b00010001;
        5'd12:  {tens, ones} = 8'b00010010;
        5'd13:  {tens, ones} = 8'b00010011;
        5'd14:  {tens, ones} = 8'b00010100;
        5'd15:  {tens, ones} = 8'b00010101;
        5'd16:  {tens, ones} = 8'b00010110;
        5'd17:  {tens, ones} = 8'b00010111;
        5'd18:  {tens, ones} = 8'b00011000;
        5'd19:  {tens, ones} = 8'b00011001;
        5'd20:  {tens, ones} = 8'b00100000;
        5'd21:  {tens, ones} = 8'b00100001;
        5'd22:  {tens, ones} = 8'b00100010;
        5'd23:  {tens, ones} = 8'b00100011;
        5'd24:  {tens, ones} = 8'b00100100;
        5'd25:  {tens, ones} = 8'b00100101;
        5'd26:  {tens, ones} = 8'b00100110;
        5'd27:  {tens, ones} = 8'b00100111;
        5'd28:  {tens, ones} = 8'b00101000;
        5'd29:  {tens, ones} = 8'b00101001;
        5'd30:  {tens, ones} = 8'b00110000;
        5'd31:  {tens, ones} = 8'b00110001;
        default: {tens, ones} = 8'b00000000;
    endcase
end

// Assign the output
assign bcd_output = {tens, ones};

endmodule
'''
filename = "binary_to_bcd/binary_to_bcd.v"
# Write the extracted Verilog code to the file
with open(filename, "w") as f:
    f.write(output_verilog_code)

Use iverilog to verify the correctness of LLM generated verilog

In [ ]:
!cd binary_to_bcd/ && iverilog -g2012 -o binary_to_bcd.vvp binary_to_bcd.v binary_to_bcd_tb.v && vvp binary_to_bcd.vvp

Testing Binary-to-BCD Converter...
VCD info: dumpfile my_design.vcd opened for output.
All test cases passed!
